In [46]:
import abc
import collections

import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneOut
import sklearn



In [28]:
cancer_dataset = pd.read_csv("datasets/cancer.csv")
cancer_labels = cancer_dataset['label'].values
cancer_points = cancer_dataset.drop(['label'], axis=1).values


In [29]:
spam_dataset = pd.read_csv("datasets/spam.csv")
spam_labels = spam_dataset['label'].values
spam_points = spam_dataset.drop(['label'], axis=1).values


In [30]:
class NNClassifier:
    def __init__(self, data, data_labeled, distance=lambda x, y: np.linalg.norm(x - y)):
        self.data_indexed = list(enumerate(data))
        self.distance = distance
        self.data_labeled = data_labeled

    def get_neighbours(self, x):
        return sorted(map(lambda tup: (tup[0], self.distance(x, tup[1])), self.data_indexed),
                      key=lambda tup: tup[1])

    @abc.abstractmethod
    def label(self, x, neighbours=None, params=None):
        return


In [31]:
class KNN(NNClassifier):
    def label(self, x, neighbours=None, k=5):
        if neighbours is None:
            neighbours = self.get_neighbours(x)

        k_dists = neighbours[:k]

        saw_labels = collections.defaultdict(lambda: 0)
        for i, x in k_dists:
            saw_labels[self.data_labeled[i]] += 1
        return sorted(saw_labels.items(), key=lambda tup: tup[1])[-1][0]


In [32]:
class RNN(NNClassifier):
    def label(self, x, neighbours=None, radius=1):
        if neighbours is None:
            neighbours = self.get_neighbours(x)

        k_dists = filter(lambda tup: tup[1] < radius, neighbours)

        saw_labels = collections.defaultdict(lambda: 0)
        for i, x in k_dists:
            saw_labels[self.data_labeled[i]] += 1
        if len(saw_labels) == 0:
            return self.data_labeled[0]
        return sorted(list(saw_labels.items()), key=lambda tup: tup[1])[-1][0]


In [33]:
def count_loo(points, labels, nnclassifier, range, loo=LeaveOneOut()):
    res_dict = collections.defaultdict(lambda: 0)
    total = 0
    for train_ind, test_ind in loo.split(points):
        points_train = points[train_ind]
        point_test = points[test_ind]
        labels_train = labels[train_ind]
        label_test = labels[test_ind]

        classifier = nnclassifier(points_train, labels_train)

        neighbours = classifier.get_neighbours(point_test)

        for val in range:
            label = classifier.label(point_test, neighbours, val)
            if label == label_test[0]:
                res_dict[val] += 1
        total += 1

    for value, count in sorted(res_dict.items()):
        print(f"{value} : {count / total}")


Execute LOO on KNN for Cancer dataset with range [1, 10]

In [34]:
print(count_loo(cancer_points, cancer_labels, KNN, list(range(1, 11))))


1 : 0.9156414762741653
2 : 0.9156414762741653
3 : 0.9261862917398945
4 : 0.9279437609841827
5 : 0.9332161687170475
6 : 0.9314586994727593
7 : 0.9314586994727593
8 : 0.9332161687170475
9 : 0.9332161687170475
10 : 0.9349736379613357
None


Execute LOO on KNN for Spam dataset with range [1, 10]

In [36]:
print(count_loo(spam_points, spam_labels, KNN, list(range(1, 11))))


1 : 0.830471636600739
2 : 0.7767876548576397
3 : 0.8148228645946534
4 : 0.7867854814170833
5 : 0.8141708324277331
6 : 0.7830906324712019
7 : 0.8041730058682895
8 : 0.783307976526842
9 : 0.797217996087807
10 : 0.780482503803521
None


To decrease time of calculation we will do kind of a binary search -- few queries with smaller and smaller ranges and steps in it to find the best radius. We will start with 1000 as maximum cause previous calculations showed that radius bigger tends to decrease quality

Execute LOO on RNN for Cancer dataset with range [1, 1000] and step 50

In [37]:
print(count_loo(cancer_points, cancer_labels, RNN, list(range(1, 1000, 50))))


1 : 0.37258347978910367
51 : 0.9437609841827768
101 : 0.9209138840070299
151 : 0.9191564147627417
201 : 0.9121265377855887
251 : 0.9103690685413005
301 : 0.8998242530755711
351 : 0.8910369068541301
401 : 0.8892794376098418
451 : 0.8840070298769771
501 : 0.8769771528998243
551 : 0.8664323374340949
601 : 0.8681898066783831
651 : 0.859402460456942
701 : 0.8541300527240774
751 : 0.8523725834797891
801 : 0.836555360281195
851 : 0.8242530755711776
901 : 0.8189806678383128
951 : 0.8066783831282952
None


In [38]:
print(count_loo(cancer_points, cancer_labels, RNN, list(range(40, 100, 5))))


40 : 0.929701230228471
45 : 0.9349736379613357
50 : 0.9420035149384886
55 : 0.9384885764499121
60 : 0.9349736379613357
65 : 0.9349736379613357
70 : 0.9349736379613357
75 : 0.929701230228471
80 : 0.9209138840070299
85 : 0.9244288224956063
90 : 0.9226713532513181
95 : 0.9226713532513181
None


In [39]:
print(count_loo(cancer_points, cancer_labels, RNN, list(range(45, 55, 1))))


45 : 0.9349736379613357
46 : 0.9367311072056239
47 : 0.9367311072056239
48 : 0.9384885764499121
49 : 0.9402460456942003
50 : 0.9420035149384886
51 : 0.9437609841827768
52 : 0.9437609841827768
53 : 0.9420035149384886
54 : 0.9384885764499121
None


In [40]:
print(count_loo(spam_points, spam_labels, RNN, list(range(1, 1000, 100))))

1 : 0.4744620734622908
101 : 0.7044120843294935
201 : 0.6822429906542056
301 : 0.6759400130406433
401 : 0.6687676592045207
501 : 0.6589871767007173
601 : 0.6572484242555966
701 : 0.6563790480330363
801 : 0.6507281025863942
901 : 0.650076070419474
None


In [42]:
print(count_loo(spam_points, spam_labels, RNN, list(range(1, 200, 20))))

1 : 0.4744620734622908
21 : 0.744620734622908
41 : 0.7298413388393827
61 : 0.7241903933927407
81 : 0.7078895892197349
101 : 0.7044120843294935
121 : 0.7007172353836123
141 : 0.6985437948272115
161 : 0.6952836339926103
181 : 0.6876765920452076
None


In [42]:
print(count_loo(spam_points, spam_labels, RNN, list(range(1, 50, 5))))

1 : 0.4744620734622908
6 : 0.763747011519235
11 : 0.7489676157357096
16 : 0.7450554227341882
21 : 0.744620734622908
26 : 0.7328841556183439
31 : 0.7254944577265812
36 : 0.7259291458378614
41 : 0.7298413388393827
46 : 0.7322321234514236
None


In [43]:
print(count_loo(spam_points, spam_labels, RNN, list(range(1, 10, 1))))

1 : 0.4744620734622908
2 : 0.5229297978700282
3 : 0.6329058900239078
4 : 0.7135405346663769
5 : 0.7604868506846337
6 : 0.763747011519235
7 : 0.7594001304064334
8 : 0.7620082590741143
9 : 0.7535318409041513
None


In [48]:
scaler = sklearn.preprocessing.MinMaxScaler()

In [49]:

norm_cancer_points = scaler.fit_transform(cancer_points)


In [50]:
print(count_loo(norm_cancer_points, cancer_labels, KNN, list(range(1, 11))))

1 : 0.9525483304042179
2 : 0.9578207381370826
3 : 0.9701230228471002
4 : 0.9648506151142355
5 : 0.9666080843585237
6 : 0.968365553602812
7 : 0.9701230228471002
8 : 0.9701230228471002
9 : 0.9701230228471002
10 : 0.968365553602812
None


In [52]:
norm_spam_points = scaler.fit_transform(spam_points)


In [53]:
print(count_loo(norm_spam_points, spam_labels, KNN, list(range(1, 11))))

1 : 0.9121930015214084
2 : 0.8821995218430776
3 : 0.9052379917409259
4 : 0.8850249945663986
5 : 0.9048033036296458
6 : 0.8876331232340795
7 : 0.9006737665724842
8 : 0.8889371875679201
9 : 0.8980656379048033
10 : 0.88871984351228
None


In [56]:
print(count_loo(norm_cancer_points, cancer_labels, RNN, np.arange(0.0, 1.0, 0.05)))

0.0 : 0.37258347978910367
0.05 : 0.37258347978910367
0.1 : 0.37258347978910367
0.15000000000000002 : 0.37961335676625657
0.2 : 0.4973637961335677
0.25 : 0.6766256590509666
0.30000000000000004 : 0.7978910369068541
0.35000000000000003 : 0.8646748681898067
0.4 : 0.9121265377855887
0.45 : 0.9209138840070299
0.5 : 0.929701230228471
0.55 : 0.9367311072056239
0.6000000000000001 : 0.929701230228471
0.65 : 0.9367311072056239
0.7000000000000001 : 0.9314586994727593
0.75 : 0.9279437609841827
0.8 : 0.9191564147627417
0.8500000000000001 : 0.9121265377855887
0.9 : 0.9015817223198594
0.9500000000000001 : 0.8875219683655536
None


In [58]:
print(count_loo(norm_cancer_points, cancer_labels, RNN, np.arange(0.5, 0.75, 0.01)))

0.5 : 0.929701230228471
0.51 : 0.9279437609841827
0.52 : 0.9367311072056239
0.53 : 0.9367311072056239
0.54 : 0.9367311072056239
0.55 : 0.9367311072056239
0.56 : 0.9402460456942003
0.5700000000000001 : 0.9367311072056239
0.5800000000000001 : 0.9332161687170475
0.5900000000000001 : 0.9349736379613357
0.6000000000000001 : 0.929701230228471
0.6100000000000001 : 0.929701230228471
0.6200000000000001 : 0.9332161687170475
0.6300000000000001 : 0.9349736379613357
0.6400000000000001 : 0.9332161687170475
0.6500000000000001 : 0.9367311072056239
0.6600000000000001 : 0.9367311072056239
0.6700000000000002 : 0.9367311072056239
0.6800000000000002 : 0.9367311072056239
0.6900000000000002 : 0.9367311072056239
0.7000000000000002 : 0.9314586994727593
0.7100000000000002 : 0.9314586994727593
0.7200000000000002 : 0.929701230228471
0.7300000000000002 : 0.929701230228471
0.7400000000000002 : 0.9279437609841827
None


In [59]:
print(count_loo(norm_spam_points, spam_labels, RNN, np.arange(0.0, 1.0, 0.05)))

0.0 : 0.39404477287546186
0.05 : 0.5646598565529233
0.1 : 0.6681156270376005
0.15000000000000002 : 0.7483155835687894
0.2 : 0.7959139317539665
0.25 : 0.8059117583134101
0.30000000000000004 : 0.7950445555314062
0.35000000000000003 : 0.7811345359704412
0.4 : 0.7426646381221473
0.45 : 0.7091936535535753
0.5 : 0.6828950228211258
0.55 : 0.6622473375353184
0.6000000000000001 : 0.63877417952619
0.65 : 0.6259508802434254
0.7000000000000001 : 0.6189958704629428
0.75 : 0.6135622690719409
0.8 : 0.6118235166268202
0.8500000000000001 : 0.608346011736579
0.9 : 0.6070419474027385
0.9500000000000001 : 0.6072592914583786
None


In [61]:
print(count_loo(norm_spam_points, spam_labels, RNN, np.arange(0.2, 0.3, 0.01)))

0.2 : 0.7959139317539665
0.21000000000000002 : 0.8015648772006085
0.22000000000000003 : 0.7991740925885678
0.23000000000000004 : 0.799826124755488
0.24000000000000005 : 0.8041730058682895
0.25000000000000006 : 0.8059117583134101
0.26000000000000006 : 0.8074331667028907
0.2700000000000001 : 0.8022169093675288
0.2800000000000001 : 0.8004781569224082
0.2900000000000001 : 0.7993914366442078
None
